In [ ]:
from sys import stdout
from openmm.app import *
from openmm import *
from simtk.unit import *

import openmmtools as tools
import MDAnalysis as md
import nglview as ng
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
### 1. Choose a water model
h2o = "spce"
# h2o = "tip3p"
# h2o = "tip4p"
# h2o = "tip4pew"
# h2o = "tip5p"

### 2. Create initial coordinates and system
water = tools.testsystems.WaterBox(model=h2o,
                                  box_edge=2.5*nanometers)

### 2a. Save initial coordinates to file
output = open('water.pdb', 'w')
PDBFile.writeFile(water.topology, water.positions, output)
output.flush()

In [ ]:
### 3. Choose parameters of the experiment: temperature, pressure and length
temperature = 320*kelvin
pressure = 1*atmosphere
simulationTime = 5*picosecond

### 3a. Simulation timestep, total number of steps and thermostat/barostat parameters
timeStep = 0.002*picoseconds
totalSteps = simulationTime / timeStep
frictionCoeff = 1/picosecond
nupdt = 25

### 4. Choose an integrator
integrator = LangevinIntegrator( temperature, frictionCoeff, timeStep )
water.system.addForce(MonteCarloBarostat( pressure , temperature , nupdt ) )

### 5 Create a simulation object
simulation = Simulation(water.topology, water.system, integrator)
simulation.context.setPositions(water.positions)
simulation.context. setVelocitiesToTemperature(temperature, 12345)

### 6. Set output for trajectory and thermodynamics data
simulation.reporters.append(DCDReporter('traj.dcd', 100))

### 7. Set output for thermodynamics data
simulation.reporters.append(StateDataReporter( "thermo.csv"
                                             , 10
                                             , step            = True
                                             , time            = True
                                             , potentialEnergy = True
                                             , temperature     = True
                                             , density         = True
                                             , volume          = True
                                             , separator       = ","
                                             ))


### 7a. Set screen output to monitor the simulation progress
simulation.reporters.append(StateDataReporter( stdout
                                             , int(totalSteps/10)
                                             , step            = True
                                             , progress        = True
                                             , remainingTime   = True
                                             , elapsedTime     = True
                                             , separator       = " "
                                             , totalSteps      = totalSteps
                                             ))


In [ ]:
### 8. Run MD
simulation.step(totalSteps)

In [ ]:
### 9. Visualization
sys = md.Universe("water.pdb", 'traj.dcd')
view = ng.show_mdanalysis(sys, gui=True)
view.center()
view.representations = [
    {"type": "ball+stick", "params": {"sele": "all"}}
]
view

In [ ]:
### 10. Load the thermodynamic data in a dataframe
data = pd.read_csv("thermo.csv")
print(data.head())
print(data.tail())

In [ ]:
x = data["Time (ps)"]
y = data["Density (g/mL)"]
plt.scatter(x,y)

xmin = np.min(x)
xmax = np.max(x)
fit = np.polyfit(x,y,1)
print("Slope              :",fit[0])
print("Intercept          :",fit[1])
# print("Drift              :",fit[0]*(xmax-xmin))

In [ ]:
average = np.mean(data["Density (g/mL)"])
stderr = np.std(data["Density (g/mL)"]) / np.sqrt(np.size(data["Density (g/mL)"]))
print(average,stderr)